<a href="https://colab.research.google.com/github/ChVenkatSai/Sampling-for-LLM/blob/main/Custom_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", padding_side="left")




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 47.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# Tokenize dataset
class GPT2Dataset(Dataset):
    def __init__(self, tokenizer, dataset, num_samples, split="train"):
        self.examples = tokenizer(dataset[split]["text"][:num_samples], padding=True, truncation=True, return_tensors="pt")

    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.examples.items()}

In [ ]:
from torch.cuda.amp import autocast, GradScaler
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from torch.optim.lr_scheduler import LambdaLR
import numpy as np
from torch.optim.lr_scheduler import _LRScheduler
import torch.nn.functional as F

In [ ]:
lora_config = LoraConfig(
    r=8,  # Set the rank factor
    lora_alpha=16,  # Set the alpha factor
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
    init_lora_weights="gaussian",  # Initialize LoRA weights using Gaussian distribution
    target_modules=["lm_head"]  # Specify the target attention module for LoRA
)



In [ ]:
train_dataset = GPT2Dataset(tokenizer, dataset, 1001)
test_dataset = GPT2Dataset(tokenizer, dataset, 101, "test")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Linear,  torch.nn.Conv1d)):
        print(name)
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

lm_head


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 408,200 || all params: 82,320,776 || trainable%: 0.49586510214626744


In [ ]:
batch_size = 2
epochs = 2
learning_rate = 0.01
data_size = len(train_dataset)
num_batch = data_size//batch_size + 1
T = epochs*num_batch
M = 2
temperature = 1.0
alpha = 1.0
weight_decay = 5e-4

In [ ]:
class CosineLR(_LRScheduler):
    def __init__(self, optimizer, T, M, lr_0, num_batch, last_epoch=-1):
        self.T = T
        self.M = M
        self.lr_0 = lr_0
        self.num_batch = num_batch
        super(CosineLR, self).__init__(optimizer, last_epoch)

    def step(self, epoch=None, batch_idx=None):
        if epoch is None or batch_idx is None:
              print('Breaking')
              return
        rcounter = epoch*self.num_batch+batch_idx
        cos_inner = np.pi * (rcounter % (self.T // self.M))
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        for param_group in self.optimizer.param_groups:
            lr = 0.5 * (cos_out) * self.lr_0
            param_group['lr'] = lr

In [ ]:
temperature = 1.0
alpha = 1.0

In [ ]:
import torch.optim as optim

class CustomSGD(optim.Optimizer):
    def __init__(self, params, lr=0.01, weight_decay=0, alpha=0.9, temperature=1.0, datasize=1, device_id='cuda', scheduler=None):
        defaults = dict(lr=lr, weight_decay=weight_decay, alpha=alpha, temperature=temperature, datasize=datasize, device_id=device_id)
        super(CustomSGD, self).__init__(params, defaults)

    def step(self, toperform=-torch.inf, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            weight_decay = group['weight_decay']
            alpha = group['alpha']
            temperature = group['temperature']
            datasize = group['datasize']
            device_id = group['device_id']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if not hasattr(p,'buf'):
                  p.buf = torch.zeros(p.size()).cuda(device_id)
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                buf_new = (1 - alpha) * p.buf - lr * d_p
                if toperform>0:
                    eps = torch.randn(p.size()).to(device_id)
                    buf_new += (2.0 * lr * alpha * temperature / datasize) ** 0.5 * eps
                p.data.add_(buf_new)
                p.buf = buf_new

        return loss


In [ ]:
adam = False

In [ ]:

# Optimizer
if adam:
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
else:
  optimizer = CustomSGD(params=model.parameters(), lr=learning_rate,weight_decay=weight_decay, alpha=alpha, temperature=temperature, datasize=data_size)

scheduler = CosineLR(optimizer, T, M, learning_rate, num_batch)

if not adam:
  optimizer.scheduler = scheduler
# Data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Custom training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

scaler = GradScaler()



for epoch in range(epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        optimizer.zero_grad()


        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()

        toperform = batch_idx - len(train_loader) + 50

        if batch_idx >= len(train_loader) - 50:  # Add noise during the last 5 steps of each epoch
            for param in model.parameters():
                if param.grad is not None:
                    # Scale the noise magnitude
                    eps = torch.randn(param.grad.size()).to(device)
                    param.grad += (2.0 * alpha * temperature / data_size) ** 0.5 * eps

        if adam:
          optimizer.step()
        else:
          optimizer.step()
        scheduler.step(epoch, batch_idx)
        total_loss += loss.item()




        if batch_idx % 100 == 0:
            print('Batch [{}/{}], Loss: {:.3f}'.format(batch_idx + 1, len(train_loader), total_loss / (batch_idx + 1)))

    save_path = f'finetuned_model_epoch_{epoch+1}.pt'
    torch.save(model.state_dict(), save_path)

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {average_loss}")



Breaking


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
<ipython-input-10-6a12a3f790c6>:28: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1630.)
  d_p.add_(weight_decay, p.data)


Batch [1/501], Loss: 8.029
Batch [101/501], Loss: 1.444
Batch [201/501], Loss: 1.349
Batch [301/501], Loss: 1.322
Batch [401/501], Loss: 1.280
Batch [501/501], Loss: 1.244
Epoch 1/2, Average Loss: 1.2437274637083688
Batch [1/501], Loss: 2.381
Batch [101/501], Loss: 1.056
Batch [201/501], Loss: 1.175
Batch [301/501], Loss: 1.127
Batch [401/501], Loss: 1.133
Batch [501/501], Loss: 1.140
Epoch 2/2, Average Loss: 1.139564252122047


In [ ]:

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Variables to track loss
total_loss = 0
num_batches = len(test_loader)
ensemble_predictions = []
total_tokens = 0
correct_tokens = 0
total_entropy_loss = 0

# Evaluation mode with dropout enabled
model.train()  # Set model to training mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prediction loop
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Forward pass
        all_predictions = []
        batch_loss = 0
        batch_entropy_loss = 0
        for epoch in range(epochs):
          model.load_state_dict(torch.load(f'finetuned_model_epoch_{epoch+1}.pt'))
          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
          all_predictions.append(outputs.logits)
          batch_loss += outputs.loss.item()
          #batch_entropy_loss += F.cross_entropy(outputs.logits.view(-1, outputs.logits.size(-1)), input_ids.view(-1)).item()


        averaged_predictions = torch.stack(all_predictions).mean(dim=0)
        ensemble_predictions.append(averaged_predictions)

        predicted_tokens = torch.argmax(averaged_predictions, dim=-1)
        correct_tokens += (predicted_tokens == input_ids).sum().item()
        total_tokens += input_ids.numel()

        total_loss += batch_loss / epochs
        #total_entropy_loss += batch_entropy_loss / (epochs)

        if batch_idx%10 == 0:
          print('Batch [{}/{}], Loss: {:.3f}'.format(batch_idx + 1, len(test_loader), total_loss / (batch_idx + 1)))

perplexity = torch.exp(torch.tensor(total_loss / len(test_loader)))
average_loss = total_loss / (num_batches)
token_level_accuracy = correct_tokens / total_tokens
average_entropy = total_entropy_loss/len(test_loader)


# Print test loss
print(f"Test Loss: {average_loss}")
#print(f"Entropy Loss: {average_entropy}")
print(f"Perplexity: {perplexity.item()}")
print(f"Token-level Accuracy: {token_level_accuracy}")



Batch [1/51], Loss: 0.163
Batch [11/51], Loss: 1.959
Batch [21/51], Loss: 1.406
Batch [31/51], Loss: 1.706
Batch [41/51], Loss: 1.636
Batch [51/51], Loss: 1.813
Test Loss: 1.8132188064959265
Perplexity: 6.130147457122803
Token-level Accuracy: 0.8787588796054326


In [ ]:
# Evaluation mode with dropout enabled
model.eval()  # Set model to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", padding_side="left")
# Lists to store predicted token IDs for all examples
all_predicted_token_ids = []

# Prediction loop
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Forward pass
        outputs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  max_length=input_ids.shape[1] + 50,  # Adjust max_length based on input length
                                  num_return_sequences=1,
                                  pad_token_id=tokenizer.eos_token_id)  # Set pad_token_id to eos_token_id
        predicted_token_ids = outputs[0].tolist()  # Take the first generated sequence

        # Collect predicted token IDs for all examples in the batch
        all_predicted_token_ids.extend(predicted_token_ids)

# Convert token IDs to tokens
all_predicted_texts = tokenizer.batch_decode(all_predicted_token_ids, skip_special_tokens=True)

# Print predicted texts for all examples
predicted_texts_combined = " | ".join(all_predicted_texts)

# Print all predicted texts in the same line
print(predicted_texts_combined)
  # Separate examples with dashes


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 